In [1]:
cd '/Volumes/OS X EL CAPITAN/Dropbox/Pints/Pints1/pints'

/Volumes/OS X EL CAPITAN/Dropbox/Pints/Pints1/pints


In [ ]:
from __future__ import print_function
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import multiprocessing

# Runs a single iteration of Random Walk Metropolis targeting toy_log_pdf
def run_single_RWM(toy_log_pdf, num_iterations, x0):
    # Create sampler
    mcmc = pints.MCMCSampling(toy_log_pdf, 4, x0, method=pints.MetropolisRandomWalkMCMC)

    # Set maximum number of iterations
    mcmc.set_max_iterations(num_iterations)

    # Disable logging
    mcmc.set_log_to_screen(False)

    # Number of chains
    num_chains = 4
    
    # Run!
    chains = mcmc.run()
    
    # Discard warm up
    chains = chains[:, int(num_iterations / 2.0):, :]
    
    # Calculate covariance matrix
    result_object = pints._mcmc._result.McmcResultObject(chains)
    cov_est = np.cov(result_object.extract_all().T)
    
    # Compare with true
    cov_true = log_pdf.covariance_matrix()
    error = np.linalg.norm(toy_log_pdf.covariance_matrix() - cov_est)
    
    return error

def wrapper(i):
    return run_single_RWM(log_pdf, 8000, lX0[i])

num_iterates = 10
num_cores = 10
inputs = np.arange(num_iterates)

# Create X0 outside of parallelisation otherwise joblib uses same seed in all child processes
def create_x0(num_iterates, dimension, extremum):
    lX0 = []
    for i in range(num_iterates):
        lX0.append(np.random.uniform(np.full(dimension, -1*extremum), np.full(dimension, extremum),size=(4, dimension)))
    return lX0

# 3D normal with correlation between dimensions of 0.5

In [ ]:
log_pdf = toy.HighDimensionalNormalLogPDF(dimension=3,correlation=0.5)
lX0 = create_x0(num_iterates, 3, 8)
results_3_low = Parallel(n_jobs=num_cores)(delayed(wrapper)(i) for i in inputs)
plt.hist(results_3_low)
plt.xlabel('Error')
plt.ylabel('Count')
plt.show()
np.savetxt("RWM_error_3d_normal_low.csv", results_3_low, delimiter=",")

# 3D normal with correlation = 0.95

In [ ]:
log_pdf = toy.HighDimensionalNormalLogPDF(dimension=3,correlation=0.95)
lX0 = create_x0(num_iterates, 3, 8)
results_3_high = Parallel(n_jobs=num_cores)(delayed(wrapper)(i) for i in inputs)
plt.hist(results_3_high)
plt.xlabel('Error')
plt.ylabel('Count')
plt.show()
np.savetxt("RWM_error_3d_normal_high.csv", results_3_high, delimiter=",")

# 6D normal with correlation = 0.5

In [ ]:
log_pdf = toy.HighDimensionalNormalLogPDF(dimension=6,correlation=0.5)
lX0 = create_x0(num_iterates, 6, 8)
results_6_low = Parallel(n_jobs=num_cores)(delayed(wrapper)(i) for i in inputs)
plt.hist(results_6_low)
plt.xlabel('Error')
plt.ylabel('Count')
plt.show()
np.savetxt("RWM_error_6d_normal_low.csv", results_6_low, delimiter=",")

# 6D normal with correlation = 0.95

In [ ]:
log_pdf = toy.HighDimensionalNormalLogPDF(dimension=6,correlation=0.95)
lX0 = create_x0(num_iterates, 6, 8)
results_6_high = Parallel(n_jobs=num_cores)(delayed(wrapper)(i) for i in inputs)
plt.hist(results_6_high)
plt.xlabel('Error')
plt.ylabel('Count')
plt.show()
np.savetxt("RWM_error_6d_normal_high.csv", results_6_high, delimiter=",")